In [1]:
# import API KEY

import os                           # operating system library
from dotenv import load_dotenv      # load environment variables  


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai.chat_models import ChatOpenAI   # LangChain connection to OpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-turbo")

response = model.invoke("What is the Suez Canal?")

response.content

"The Suez Canal is a man-made waterway in Egypt, connecting the Mediterranean Sea to the Red Sea. This canal is one of the world's most significant shipping routes, allowing ships to travel between Europe and South Asia without navigating around Africa, thereby significantly reducing travel time and distance.\n\nCompleted in 1869 under the direction of the French engineer Ferdinand de Lesseps, the canal stretches approximately 193 kilometers (120 miles). It has no locks because the sea level is the same at both ends, and it can accommodate ships of various sizes, including some of the largest vessels, such as oil tankers.\n\nThe Suez Canal is strategically and economically crucial, as it is a major conduit for the transport of oil, as well as other goods. It has also been a focal point in geopolitical conflicts. For instance, it was nationalized by Egypt's President Gamal Abdel Nasser in 1956, leading to the Suez Crisis involving Egypt, Israel, France, and the United Kingdom. The canal

In [3]:
os.listdir("../pdfs")

['2024_BRIEFING.pdf']

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [5]:
loader = PyPDFDirectoryLoader("../pdfs/")

pages = loader.load()

In [6]:
len(pages)

8

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings


vectorizer = OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(pages, vectorizer, persist_directory="../chroma_db")

In [9]:
retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "lambda_mult": 0.25})

In [10]:
from langchain.prompts import ChatPromptTemplate

In [11]:
template = """
            Given the context below and the question, 
            please generate a header, and 5 bullet points, Summarize each bullet point in 40 words.
            Also fetch sub_titles and numbers to describe the information.

            Context: {context}

            Question: {question}
            """


prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
from langchain_core.runnables import RunnablePassthrough

In [14]:
query = "What are the endnotes of the briefing?"

In [15]:
in_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | parser


response = in_chain.invoke(query)

In [16]:
response.split('\n')

['## Impact of Recent Threats in the Red Sea on Regional and EU Economies',
 '',
 '### 1. Disruption of Global Supply Chains',
 '- The attacks by the Houthi militia near the Bab el-Mandeb Strait led to the suspension of Suez transit by major shipping companies, causing global supply chain disruptions. This situation forced some companies to halt production due to delays in shipping and increased freight costs.',
 '',
 '### 2. Economic Consequences of Shipping Diversion',
 '- Shipping diversions to the Cape of Good Hope route have extended travel times by 10 to 14 days, escalating shipping costs further. This increase is compounded by already rising fuel and insurance expenses, affecting overall trade costs.',
 '',
 '### 3. Impact on Energy Supplies and Inflation',
 "- Persisting or worsening disruptions could severely impact energy supplies and prices, potentially driving higher inflation rates. This scenario threatens to undermine central bank efforts aimed at stabilizing the economic

In [17]:
from langchain_openai import OpenAI

input_model = OpenAI(temperature=0, max_tokens=1024)

In [18]:
template = """
            We have provided  information below.
            Given this information, please generate python-pptx code for a single 
            slide with this information set header and subtitle as title and  
            bullet points.
            
            Put the title on top of the slide and center all text to slide size.
            
            Separate the bullet points into separate texts with line separator.
            Set font size to minimum for fixing text to cells. Save the file in ../pptx folder

            Information: {context}
            """


prompt = ChatPromptTemplate.from_template(template)

In [19]:
out_chain = prompt | input_model | parser

In [20]:
output = out_chain.invoke({"context": response})

In [21]:
output.split('\n')

['',
 '# Import necessary libraries',
 'from pptx import Presentation',
 'from pptx.util import Inches',
 '',
 '# Create new presentation',
 'prs = Presentation()',
 '',
 '# Add new slide',
 'slide = prs.slides.add_slide(prs.slide_layouts[0])',
 '',
 '# Set title and subtitle',
 'title = slide.shapes.title',
 'title.text = "Impact of Recent Threats in the Red Sea on Regional and EU Economies"',
 'subtitle = slide.placeholders[1]',
 'subtitle.text = "Disruption of Global Supply Chains, Economic Consequences of Shipping Diversion, Impact on Energy Supplies and Inflation, Risk to International Debt Repayment, Threats to Submarine Communication Cables"',
 '',
 '# Center all text',
 'title.text_frame.paragraphs[0].alignment = 1',
 'subtitle.text_frame.paragraphs[0].alignment = 1',
 '',
 '# Set font size to minimum',
 'title.text_frame.paragraphs[0].font.size = Inches(0.1)',
 'subtitle.text_frame.paragraphs[0].font.size = Inches(0.1)',
 '',
 '# Add bullet points',
 'bullet_points = slide.sha

In [22]:
exec('print(2+2)')

4


In [23]:
exec(output)